In [130]:
# go through this first: https://www.tensorflow.org/text/tutorials/classify_text_with_bert#export_for_inference

import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from helper import remove_unused_columns, transform_profession, preprocess_text
import pandas as pd
import transformers
import re
import numpy as np
from tensorflow.python.keras import layers, models
from tqdm import tqdm

tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import TaggedDocument

tf.get_logger().setLevel('ERROR')

## Clean up your data

In [131]:
# reading our files and
file = '../excel-data/all-data-linkedin.csv'
data_csv = pd.read_csv(file)
data_top = remove_unused_columns(data_csv)[0:13]

# -------- Get our columns into lists --------

headline_categories = list(data_csv['Industry'])


def encode_labels(headline) -> ([int], dict):
    categories_encode = {}
    enc_values = []

    for i, categories in enumerate(list(data_csv['Industry'])):
        if categories not in categories_encode:
            categories_encode[categories] = i

    for item in headline:
        num = categories_encode[item] if item in categories_encode else ''
        enc_values.append(num)

    return enc_values, categories_encode


profession = transform_profession(data_top, data=data_csv)
category_list = data_csv['Headline']
category_id = [i for i in range(len(category_list))]
encoded_labels, categories_label = encode_labels(headline_categories)

# Initialize our columns into a dataframe
dtf = pd.DataFrame()
dtf['category_id'] = category_id
dtf['categories'] = headline_categories
dtf['profession'] = profession
dtf['categories_encode'] = encoded_labels

# Clean your data set first remove unwanted words like: "I", "me", "you"
list_stop_of_words = stopwords.words('english')
dtf['clean_text_profession'] = dtf['profession'].apply(
    lambda x: preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=list_stop_of_words))

dtf.head()

,category_id,categories,profession,categories_encode,clean_text_profession
0,0,Machine Learning,"Director of Data Science, Machine Learning at ...",0,director data science machine learning walmart...
1,1,Machine Learning,Machine learning on Encrypted data Engineer,0,machine learning encrypted data engineer
2,2,Machine Learning,Machine Learning Research Scientist - Deep Lea...,0,machine learning research scientist deep learning
3,3,Machine Learning,Principal (Manager) R&D Data Scientist,0,principal manager rd data scientist
4,4,Machine Learning,"Vice President of Machine Learning, Merchandis...",0,vice president machine learning merchandising ...


## Split data set

In [132]:
## split dataset
dtf_train, dtf_test = train_test_split(dtf, test_size=0.3)
## get target
y_train = dtf_train['categories'].values
y_test = dtf_test['categories'].values

## Using BERT

In [133]:
txt = "bank river"

tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

nlp = transformers.TFBertModel.from_pretrained('bert-base-uncased')

input_ids = np.array(tokenizer.encode(txt))[None, :]
embedding = nlp(input_ids)

In [134]:
tokenizer = transformers.AutoTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

## Compiling the model

In [135]:
corpus = dtf['clean_text_profession']
maxlen = 50

maxqnans = np.int((maxlen - 20) / 2)
corpus_tokenized = ["[CLS] " +
                    " ".join(
                        tokenizer.tokenize(re.sub(r'[^\w\s]+|\n', '', str(txt).lower().strip()))[:maxqnans]) + " [SEP] "
                    for txt in corpus]

## generate masks
masks = [[1] * len(txt.split(" ")) + [0] * (maxlen - len(
    txt.split(" "))) for txt in corpus_tokenized]

## padding
txt2seq = [txt + " [PAD]" * (maxlen - len(txt.split(" "))) if len(txt.split(" ")) != maxlen else txt for txt in
           corpus_tokenized]

## generate idx
idx = [tokenizer.encode(seq.split(" ")) for seq in txt2seq]

## generate segments
segments = []
for seq in txt2seq:
    temp, i = [], 0
    for token in seq.split(" "):
        temp.append(i)
        if token == "[SEP]":
            i += 1
    segments.append(temp)
## feature matrix
X_train = [np.asarray(idx, dtype='int32'),
           np.asarray(masks, dtype='int32'),
           np.asarray(segments, dtype='int32')]

i = 0
print("txt: ", dtf['clean_text_profession'].iloc[0])
print("tokenized:", [tokenizer.convert_ids_to_tokens(idx) for idx in X_train[0][i].tolist()])
print("idx: ", X_train[0][i])
print("mask: ", X_train[1][i])
print("segment: ", X_train[2][i])

txt:  director data science machine learning walmart global tech
tokenized: ['[CLS]', 'director', 'data', 'science', 'machine', 'learning', 'wal', '##mart', 'global', 'tech', '[SEP]', '[UNK]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
idx:  [  101  2472  2951  2671  3698  4083 24547 22345  3795  6627   102   100
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0]
mask:  [1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
segment:  [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 

<ipython-input-135-fac01dfe15a0>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  maxqnans = np.int((maxlen - 20) / 2)


In [136]:
## inputs
idx = layers.Input(50, dtype="int32", name="input_idx")
masks = layers.Input(50, dtype="int32", name="input_masks")

## pre-trained bert with config
config = transformers.DistilBertConfig(dropout=0.2,
                                       attention_dropout=0.2)
config.output_hidden_states = False
nlp = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)

bert_out = nlp(idx, attention_mask=masks)[0]

## fine-tuning
x = layers.GlobalAveragePooling1D()(bert_out)
x = layers.Dense(64, activation="relu")(x)
y_out = layers.Dense(len(np.unique(y_train)), activation='softmax')(x)

## compile
model = models.Model([idx, masks], y_out)

for layer in model.layers[:3]:
    layer.trainable = False

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_idx (InputLayer)          [(None, 50)]         0                                            
__________________________________________________________________________________________________
tf.compat.v1.shape_89 (TFOpLamb (2,)                 0           input_idx[0][0]                  
__________________________________________________________________________________________________
tf.__operators__.getitem_98 (Sl ()                   0           tf.compat.v1.shape_89[0][0]      
__________________________________________________________________________________________________
tf.range_5 (TFOpLambda)         (50,)                0           tf.__operators__.getitem_98[0][0]
____________________________________________________________________________________________

In [147]:
## encode y
X_test = dtf_test['clean_text_profession']

dic_y_mapping = {n: label for n, label in enumerate(np.unique(y_train))}
inverse_dic = {v: k for k, v in dic_y_mapping.items()}

y_train = np.array([inverse_dic[y] for y in y_train])

# ## train
# training = model.fit(x=X_train, y=y_train, batch_size=64, epochs=1, shuffle=True, verbose=1)

## test
# predicted_prob = model.predict(X_test)
# predicted = [dic_y_mapping[np.argmax(pred)] for pred in predicted_prob]

[array([[ 101, 2472, 2951, ...,    0,    0,    0],
       [ 101, 3698, 4083, ...,    0,    0,    0],
       [ 101, 3698, 4083, ...,    0,    0,    0],
       ...,
       [ 101, 2604, 3099, ...,    0,    0,    0],
       [ 101, 6605, 4054, ...,    0,    0,    0],
       [ 101, 2472, 5160, ...,    0,    0,    0]], dtype=int32), array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32), array([[0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]], dtype=int32)]
